<a href="https://colab.research.google.com/github/rlberry-py/tutorials/blob/main/Deep_Q_Learning/Solution_Tutorial_Deep_Q_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tutorial - Deep Q-Learning 

Deep Q-Learning uses a neural network to approximate $Q$ functions. Hence, we usually refer to this algorithm as DQN (for *deep Q network*).

The parameters of the neural network are denoted by $\theta$. 
*   As input, the network takes a state $s$,
*   As output, the network returns $Q(s, a, \theta)$, the value of each action $a$ in state $s$, according to the parameters $\theta$.


The goal of Deep Q-Learning is to learn the parameters $\theta$ so that $Q(s, a, \theta)$ approximates well the optimal $Q$-function $Q^*(s, a)$. 

In addition to the network with parameters $\theta$, the algorithm keeps another network with the same architecture and parameters $\theta^-$, called **target network**.

The algorithm works as follows:

1.   At each time $t$, the agent is in state $s_t$ and has observed the transitions $(s_i, a_i, r_i, s_i')_{i=1}^{t-1}$, which are stored in a **replay buffer**.

2.  Choose action $a_t = \arg\max_a Q(s_t, a)$ with probability $1-\varepsilon_t$, and $a_t$=random action with probability $\varepsilon_t$. 

3. Take action $a_t$, observe reward $r_t$ and next state $s_t'$.

4. Add transition $(s_t, a_t, r_t, s_t')$ to the **replay buffer**.

4.  Sample a minibatch $\mathcal{B}$ containing $B$ transitions from the replay buffer. Using this minibatch, we define the loss:

$$
L(\theta) = \sum_{(s_i, a_i, r_i, s_i') \in \mathcal{B}}
\left[
Q(s_i, a_i, \theta) -  y_i
\right]^2
$$
where the $y_i$ are the **targets** computed with the **target network** $\theta^-$:

$$
y_i = r_i + \gamma \max_{a'} Q(s_i', a', \theta^-).
$$

5. Update the parameters $\theta$ to minimize the loss, e.g., with gradient descent (**keeping $\theta^-$ fixed**): 
$$
\theta \gets \theta - \eta \nabla_\theta L(\theta)
$$
where $\eta$ is the optimization learning rate. 

6. Every $N$ transitions ($t\mod N$ = 0), update target parameters: $\theta^- \gets \theta$.

7. $t \gets t+1$. Stop if $t = T$, otherwise go to step 2.

# Colab setup

In [ ]:
# After installing, restart the kernel

# install rlberry library
!git clone https://github.com/rlberry-py/rlberry.git 
!cd rlberry && git pull && pip install -e .[full] > /dev/null 2>&1
!pip install ffmpeg-python > /dev/null 2>&1

# packages required to show video
!pip install pyvirtualdisplay > /dev/null 2>&1
!apt-get install -y xvfb python-opengl ffmpeg > /dev/null 2>&1

# ask to restart runtime
print("")
print(" ~~~  Libraries installed, please restart the runtime! ~~~ ")
print("")

Cloning into 'rlberry'...
remote: Enumerating objects: 190, done.
remote: Counting objects: 100% (190/190), done.
remote: Compressing objects: 100% (142/142), done.
remote: Total 3259 (delta 93), reused 113 (delta 48), pack-reused 3069
Receiving objects: 100% (3259/3259), 834.18 KiB | 6.27 MiB/s, done.
Resolving deltas: 100% (2087/2087), done.
Already up to date.

 ~~~  Libraries installed, please restart the runtime! ~~~ 



In [ ]:
# Imports
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import random
from copy import deepcopy
from gym.wrappers import Monitor
import gym

# Random number generator
import rlberry.seeding as seeding 
rng = seeding.get_rng()

# 1. Define the parameters

In [ ]:
# Environment
env = gym.make("CartPole-v0")

# Discount factor
GAMMA = 0.99

# Batch size
BATCH_SIZE = 256
# Capacity of the replay buffer
BUFFER_CAPACITY = 10000
# Update target net every ... episodes
UPDATE_TARGET_EVERY = 20

# Initial value of epsilon
EPSILON_START = 1.0
# Parameter to decrease epsilon
DECREASE_EPSILON = 200
# Minimum value of epislon
EPSILON_MIN = 0.05

# Number of training episodes
N_EPISODES = 400

# Learning rate
LEARNING_RATE = 0.1

INFO: Making new env: CartPole-v0


# 2. Define the replay buffer

In [ ]:
class ReplayBuffer:
    def __init__(self, capacity):
        self.capacity = capacity
        self.memory = []
        self.position = 0

    def push(self, state, action, reward, next_state):
        """Saves a transition."""
        if len(self.memory) < self.capacity:
            self.memory.append(None)
        self.memory[self.position] = (state, action, reward, next_state)
        self.position = (self.position + 1) % self.capacity

    def sample(self, batch_size):
        return rng.choice(self.memory, batch_size).tolist()


    def __len__(self):
        return len(self.memory)

# create instance of replay buffer
replay_buffer = ReplayBuffer(BUFFER_CAPACITY)

# 3. Define the neural network architecture, objective and optimizer

In [ ]:
class Net(nn.Module):
    """
    Basic neural net.
    """
    def __init__(self, obs_size, hidden_size, n_actions):
        super(Net, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(obs_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, n_actions)
        )

    def forward(self, x):
        return self.net(x)


# Example: 

# state = env.observation_space.sample()
# state_torch = torch.from_numpy(state)

# net = Net(state.shape[0], 128, env.action_space.n)
# print(net)

# q_state = net(state_torch)
# print(state)
# print(q_state)

In [ ]:
# create network and target network
hidden_size = 128
obs_size = env.observation_space.shape[0]
n_actions = env.action_space.n

q_net = Net(obs_size, hidden_size, n_actions)
target_net = Net(obs_size, hidden_size, n_actions)

# objective and optimizer
objective = nn.MSELoss()
optimizer = optim.Adam(params=q_net.parameters(), lr=LEARNING_RATE)

# 4. Implement Deep Q-Learning

In [ ]:
#
#  Some useful functions
#

def get_q(states):
    """
    Compute Q function for a list of states
    """
    with torch.no_grad():
        states_v = torch.FloatTensor([states])
        output = q_net.forward(states_v).data.numpy()  # shape (1, len(states), n_actions)
    return output[0, :, :]  # shape (len(states), n_actions)

def eval_dqn(n_sim=5):
    """
    ** TO BE IMPLEMENTED **
    
    Monte Carlo evaluation of DQN agent.

    Repeat n_sim times:
        * Run the DQN policy until the environment reaches a terminal state (= one episode)
        * Compute the sum of rewards in this episode
        * Store the sum of rewards in the episode_rewards array.
    """
    env_copy = deepcopy(env)
    episode_rewards = np.zeros(n_sim)

    for ii in range(n_sim):
        state = env_copy.reset()
        done = False 
        while not done:
            action = choose_action(state, 0.0)
            next_state, reward, done, _ = env_copy.step(action)
            episode_rewards[ii] += reward
            state = next_state
    return episode_rewards

In [ ]:
def choose_action(state, epsilon):
    """
    ** TO BE IMPLEMENTED **
    
    Return action according to an epsilon-greedy exploration policy
    """
    if np.random.uniform() < epsilon:
        return env.action_space.sample()
    else:
        q = get_q([state])[0]
        return q.argmax()    

def update(state, action, reward, next_state, done):
    """
    ** TO BE COMPLETED **
    """
    
    # add data to replay buffer
    if done:
        next_state = None
    replay_buffer.push(state, action, reward, next_state)
    
    if len(replay_buffer) < BATCH_SIZE:
        return np.inf
    
    # get batch
    transitions = replay_buffer.sample(BATCH_SIZE)


    # values_debug  = torch.zeros(BATCH_SIZE)   # to be computed using batch   ->  Q(s_i, a_i)
    # targets_debug = torch.zeros(BATCH_SIZE)   # to be computed using batch   ->  y_i 

    # for ii, transition in enumerate(transitions):
    #     s_i, a_i, r_i, next_s_i = transition

    #     # compute Q(s_i, a_i) = Q_i
    #     s_i = torch.FloatTensor([s_i])[0]
    #     Q_i = q_net(s_i)[a_i]
    #     values_debug[ii] = Q_i


    #     # compute y_i
    #     r_i = torch.FloatTensor([r_i])[0]

    #     # 
    #     max_q_next_state = 0.0
    #     if next_s_i is not None:
    #         next_s_i = torch.FloatTensor([next_s_i])[0]
    #         max_q_next_state = target_net(next_s_i).detach().max()

    #     y_i = r_i + GAMMA*max_q_next_state

    #     targets_debug[ii] = y_i

    # # ----------------------------------------------

    # ------------------------
    # process batch of (state, action, reward, next_state)
    states = [transitions[ii][0] for  ii in range(BATCH_SIZE)]
    actions = [transitions[ii][1] for  ii in range(BATCH_SIZE)] 
    rewards = [transitions[ii][2] for  ii in range(BATCH_SIZE)] 

    # Attention: next_state is None when the previous state is terminal.
    # we handle this using a mask.
    next_states = [transitions[ii][3] for  ii in range(BATCH_SIZE) if transitions[ii][3] is not None ] 
    mask = [transitions[ii][3] is not None for  ii in range(BATCH_SIZE)]

    # Convert to torch tensors
    states_torch = torch.FloatTensor(states)
    actions_torch = torch.LongTensor(actions).view(-1,1)
    rewards_torch = torch.FloatTensor(rewards).view(-1, 1)
    next_states_torch = torch.FloatTensor(next_states)
    mask_torch = torch.BoolTensor(mask)

    # Q(s_i, a_i)
    values = q_net(states_torch)
    values = torch.gather(values, dim=1, index=actions_torch)

    # max_a Q(s_{i+1}, a)
    values_next_states = torch.zeros(BATCH_SIZE)
    values_next_states[mask] = target_net(next_states_torch).max(dim=1)[0].detach()
    values_next_states = values_next_states.view(-1, 1)

    # targets y_i
    targets = rewards_torch + GAMMA*values_next_states
    
    # Loss function
    loss = objective(values, targets)
     
    # Optimize the model 
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    return loss.data.numpy()

In [ ]:

#
# Train
# 

EVAL_EVERY = 5
REWARD_THRESHOLD = 199

def train():
    state = env.reset()
    epsilon = EPSILON_START
    ep = 0
    total_time = 0
    while ep < N_EPISODES:
        action = choose_action(state, epsilon)

        # take action and update replay buffer and networks
        next_state, reward, done, _ = env.step(action)
        loss = update(state, action, reward, next_state, done)

        # update state
        state = next_state

        # end episode if done
        if done:
            state = env.reset()
            ep   += 1
            if ( (ep+1)% EVAL_EVERY == 0):
                rewards = eval_dqn()
                print("episode =", ep+1, ", reward = ", np.mean(rewards))
                if np.mean(rewards) >= REWARD_THRESHOLD:
                    break

            # update target network
            if ep % UPDATE_TARGET_EVERY == 0:
                target_net.load_state_dict(q_net.state_dict())
            # decrease epsilon
            epsilon = EPSILON_MIN + (EPSILON_START - EPSILON_MIN) * \
                            np.exp(-1. * ep / DECREASE_EPSILON )    

        total_time += 1

# Run the training loop
train()

# Evaluate the final policy
rewards = eval_dqn(20)
print("")
print("mean reward after training = ", np.mean(rewards))

episode = 5 , reward =  77.4
episode = 10 , reward =  200.0

mean reward after training =  200.0


# Visualize the DQN policy

In [ ]:
from pyvirtualdisplay import Display
from IPython import display as ipythondisplay
from IPython.display import clear_output
from pathlib import Path
import base64


display = Display(visible=0, size=(1400, 900))
display.start()

def show_video():
    html = []
    for mp4 in Path("videos").glob("*.mp4"):
        video_b64 = base64.b64encode(mp4.read_bytes())
        html.append('''<video alt="{}" autoplay 
                      loop controls style="height: 400px;">
                      <source src="data:video/mp4;base64,{}" type="video/mp4" />
                 </video>'''.format(mp4, video_b64.decode('ascii')))
    ipythondisplay.display(ipythondisplay.HTML(data="<br>".join(html)))
    
env = Monitor(env, './videos', force=True, video_callable=lambda episode: True)

for episode in range(1):
    done = False
    state = env.reset()
    while not done:
        action = choose_action(state, 0.0)
        state, reward, done, info = env.step(action)
env.close()
show_video()

INFO: Clearing 4 monitor files from previous run (because force=True was provided)
INFO: Starting new video recorder writing to /content/videos/openaigym.video.3.489.video000000.mp4
INFO: Finished writing results. You can upload them to the scoreboard via gym.upload('/content/videos')
